# Bike Rentals

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from itertools import combinations

import pdpexplorer
from pdpexplorer.pdp import partial_dependence

In [ ]:
df_original = pd.read_csv('bike-sharing.csv').drop(columns=['yr'])

In [ ]:
categories = ['weathersit']
df_one_hot = pd.get_dummies(df_original, columns=categories)

In [ ]:
feature_to_one_hot = {
    original_feat: [
        (feat, feat.split('_')[-1])
        for feat in df_one_hot.columns
        if feat.startswith(original_feat)
    ]
    for original_feat in categories
}
feature_to_one_hot

In [ ]:
df_X = df_one_hot.drop(columns=['label', 'prediction'])

In [ ]:
y = df_original['label'].to_numpy()

In [ ]:
regr = RandomForestRegressor(n_estimators=20)
regr.fit(df_X, y)

In [ ]:
features = [col for col in df_original.columns if col != 'label' and col != 'prediction']
pairs = list(combinations(features, 2))

In [ ]:
pd_data = partial_dependence(
    predict=regr.predict,
    df=df_X,
    one_way_features=features,
    two_way_feature_pairs=pairs,
    one_hot_features=feature_to_one_hot,
    n_instances=1000,
    resolution=20,
    n_jobs=1,
)

In [ ]:
w = pdpexplorer.PDPExplorerWidget(
    predict=regr.predict,
    df=df_X,
    one_hot_features=feature_to_one_hot,
    pd_data=pd_data,
    n_jobs=8,
    height=650
)

w

In [ ]:
mnth = [x for x in w.single_pdps if x['id'] == 'mnth'][0]
mnth

In [ ]:
[cluster['rules'] for cluster in mnth['ice']['clusters']]